# Manipulation avancée d’expression avec `SymPy`

Dans cette section, nous allons voir quelques techniques pour réaliser des manipulations d’expressions plus avancées.

## Comprendre l’arbre des expressions

Avant de rentrer dans le vif du sujet, nous devons comprendre comment les expressions sont représentées dans `SymPy`.
Une expression mathématique est représentée par un arbre.

Prenons par exemple $x^2+xy$, ie `x**2 + x*y`.

Nous pouvons voir à quoi cette expression ressemble "sous le capot" en utilisant la fonction `srepr()`.

In [ ]:
from sympy import *
x, y, z = symbols('x y z')

In [ ]:
expr = x**2 + x*y
expr

In [ ]:
srepr(expr)

Le plus simple pour décortiquer tout ça est de regarder ce diagramme de l’arbre de notre expression :

![](https://docs.sympy.org/latest/_images/graphviz-d6cfeb1f975b9a36682b7d430767a2c103d8e138.svg)

Tout d’abord, regardons les feuilles de notre arbre. Les symboles sont des instances de la classe `Symbol`. Qu’on utilise la fonction `symbols()` ou qu'on utilise le constructeur de classe `Symbol()`, cela revient au même.


In [ ]:
x = symbols('x')

idem que 

In [ ]:
x = Symbol('x')

Dans les deux cas, on obtient un objet `Symbol` avec le nom `'x'`. Pour le nombre dans notre expression, 2, il est devenu `Integer(2)`. `Integer` est la classe `SymPy` pour les entiers.
Il est similaire au type de base Python `int`, mais il est conçu pour se combiner correctement avec les autres objets `SymPy`.

Quand on a fait `x**2`, on a créé un objet de la classe `Pow`, soit une puissance.

In [ ]:
srepr(x**2)

On aurait pu créer le même objet directement avec le constructeur de classe `Pow()`

In [ ]:
Pow(x, 2)

Notez que dans le résultat de `srepr(x**2)`, on voit `Integer(2)`, même si techniquement, c'est un 2 `int` Python qu'on tappe.

De manière générale, quand on réalise des opérations entre des objets `SymPy` et des objets numériques basiques de Python, les objets basiques comme des `int` sont convertis automatiquement en objets `SymPy` avant de réaliser l’opération.

La fonction qui fait ça sous le capot est `sympify()`.

In [ ]:
type(2)

In [ ]:
type(sympify(2))

Nous avons vu que `x**2` revient à l’objet issu de `Pow(x, 2)`. 

Et `x * y` ? La classe qui représente un produit dans `SymPy` est `Mul`.

In [ ]:
srepr(x*y)

Et donc nous pourrions créer le même objet avec le constructeur de classe `Mul()`.

In [ ]:
Mul(x, y)

Enfin, nous arrivons à notre expression finale, `x**2 + x*y`. C'est la somme des deux objets précédents, `Pow(x, 2)` et `Mul(x, y)`.

La classe `SymPy` pour l’addition est `Add`, et donc, comme vous pouvez le deviner, on peut créer un objet de cette classe avec le constructeur de classe `Add()`.

L’expression complète revient à `Add(Pow(x, 2), Mul(x, y))`.

In [ ]:
Add(Pow(x, 2), Mul(x, y))

![](https://docs.sympy.org/latest/_images/graphviz-d6cfeb1f975b9a36682b7d430767a2c103d8e138.svg)

Les arbres-expressions `SymPy` peuvent avoir de nombreuses branches, et être très large où très profond. Voici un exemple plus complexe.

In [ ]:
expr = sin(x*y)/2 - x**2 + 1/y
expr

In [ ]:
srepr(expr)

Voici le diagramme correspondant :

![](https://docs.sympy.org/latest/_images/graphviz-54ddf0a95326d32e84e72798288dc3762214049f.svg)

Cette expression révèle des choses intéressantes à propos des expressions `SymPy` :

Pour `x**2`, comme prévu, nous trouvons `Pow(x, 2)`. Un niveau plus haut, nous voyons que nous avons `Mul(-1, Pow(x, 2)`. Il n’y a pas de classe pour représenter la soustraction dans `SymPy`. $x -y$ est traité comme $x + -y$, ou plus précisément $x + -1 \times y$, soit en terme de classes `SymPy` : `Add(x, Mul(-1, y))`.

In [ ]:
srepr(x - y)

![](https://docs.sympy.org/latest/_images/graphviz-5db6e084b5324a156050a50f98fd32d5de3830b5.svg)

Ensuite, intéressons-nous à `1/y`. On pourrait s’attendre à quelque chose comme `Div(1, y)`, mais comme pour la 
soustraction, il n’y a pas de classe `SymPy` représentant la division.

À la place, la division est représentée par la puissance -1. D’où le `Pow(y, -1)`. 

Et si nous avions divisé autre chose que 1 par `y`, comme `x/y` ? 

In [ ]:
expr = x/y
srepr(expr)

![](https://docs.sympy.org/latest/_images/graphviz-bdb16b2f7e291438d35e7ae6af6ca802a803b28c.svg)

`x/y` est donc représenté comme `x*y**-1`, soit `Mul(x, Pow(y, -1))`.

Enfin, regardons le terme `sin(x*y)/2`. Tout comme dans l’exemple précédent, on pourrait s'attendre à avoir
`Mul(sin(x*y), Pow(Integer(2), -1))`. Mais à la place, nous avons `Mul(Rational(1, 2), sin(x*y))`.

Les fractions sont toujours combinées à un terme unique dans une multiplication, ce qui fait que quand on divise par 2, c'est représenté par une multiplication par $\frac{1}{2}$.

Enfin, un dernier commentaire. Vous avez peut-être remarqué que l’ordre dans lequel nous avons saisi notre expression et l’ordre dans lequel elle est sortie de `srepr` ou dans le diagramme étaient différents. Vous avez peut-être constaté ce phénomène à un autre moment. Par exemple :

In [ ]:
1 + x

La raison est que les arguments d’une opération commutative comme l’addition ou la multiplication sont stockés dans un ordre arbitraire mais stable, indépendamment de l’ordre de saisie par l’utilisateur.

Pour les multiplications non-commutatives, vous pouvez créer un symbole non-commutatif avec `Symbol('A', commutative=False)`, et dans ce cas l’ordre dans une multiplication respectera l’ordre de la saisie.

De plus comme nous allons le voir dans la prochaine section, l’ordre d’affichage et l’ordre dans lequel les éléments sont stockés "sous le capot" ne sont pas nécessairement les mêmes.

En général, une chose importante à retenir quand on travaille sur des arbres d’expressions est que la représentation interne d’une expression et son affichage ne sont pas les mêmes. Et c'est vrai aussi pour la version "saisie".
Si un algorithme de manipulation ne fonctionne pas comme vous voulez, il est probable que la représentation interne de l’objet est différente de ce que vous pensiez.

## Traverser un arbre d’expression par récurrence

Maintenant que nous savons comment fonctionnent les arbres d’expression `SymPy`, regardons comment nous pouvons parcourir un tel arbre.

Chaque objet `SymPy` a deux attributs importants : `.func` et `.args`.

### `.func`

`.func` est la tête d’un objet. Par exemple, `(x*y).func`, c'est `Mul`.  
Généralement, c'est la classe d’un objet (mais il existe des exceptions à cette règle).

Deux remarques à propos de `.func`.

Premièrement, La classe d’un objet n’est pas forcément la classe qui a été utilisée pour l’instancier. Par exemple :

In [ ]:
expr = Add(x, x)
expr.func

Nous avons créé notre expression avec le constructeur de classe `Add()`, donc on pourrait s’attendre à ce que 
`expr.func` soit `Add`, mais à la place on a un `Mul`. Pourquoi ? Regardons ce qu’est devenue notre expression :

In [ ]:
expr

In [ ]:
x + x

`Add(x, x)`, ou encore `x + x` sont automatiquement convertis en `Mul(2, x)`, donc un objet `Mul`. 

Pour les plus avancés en programmation orientée objet parmi vous, les classes `SymPy` utilisent beacoup le constructeur de classe `__new__`, qui contrairement à `__init__`, permet qu’une classe différente soit retournée par le contructeur.

Deuxièmement, certaines classes ont des cas particuliers, généralement pour des raisons de performance et d’efficacité.

In [ ]:
Integer(2).func # cas général Integer

In [ ]:
Integer(0).func # cas particulier zéro

In [ ]:
Integer(1).func # cas particulier 1

In [ ]:
Integer(-1).func # cas particulier -1

En général, ces cas particuliers ne vont pas nous gêner. Les cas particuliers comme `Zero`, `One`, `NegativeOne` etc sont des sous-classes de `Integer`, donc tant que nous utilisons `isinstance`, tout ira bien.

In [ ]:
zero = Integer(0)

In [ ]:
zero.func

In [ ]:
isinstance(zero, Integer)

### `.args`

`.args` stocke les arguments de haut-niveau de l’objet. `(x*y).args` serait `(x, y)`. Regardons quelques exemples :


In [ ]:
expr = 3*y**2*x
expr

In [ ]:
expr.func

In [ ]:
expr.args

On peut en conclure que `expr == Mul(3, y**2, x)`.

Plus généralement, nous pouvons complètement reconstruire une expression à partir de ses attributs `.func` et `.args`.

In [ ]:
expr.func(*expr.args)

In [ ]:
expr == expr.func(*expr.args)

Remarquez que, bien que nous ayons saisi `3*y**2*x`, l’attribut `.args` est `(3, x, y**2)`. Dans une `Mul`, le coefficient rationnel sera mis en premier dans `args`, mais à part ça, le reste n’a pas de règle particulière, mais il y a bien un ordre.

In [ ]:
expr = y**2*3*x
expr.args

Les `args` sont ordonnés, de manière à ce que des `Mul` équivalents aient les mêmes `args`.

Cet ordre est basé sur des critères conçus pour rendre l’ordre unique et efficace, mais il n’a pas de signification mathématique.

La forme donnée par `srepr()` à notre `expr` est `Mul(3, x, Pow(y, 2))`. Si nous voulons obtenir les `.args` de `Pow(y, 2)`, il suffit d’observer que `y**2` c'est retrouvé en troisième place dans `expr.args`, *ie* `expr.args[2]`, ou encore `expr.args[-1]`.

In [ ]:
expr.args[2].args

In [ ]:
expr.args[-1].args

Creusons un peu plus loin. Quels sont les `.args` de `y` ? Ou de `2` ?

In [ ]:
y.args

In [ ]:
Integer(2).args

Les deux ont un `.args` vide. Dans `SymPy`, un `.args` vide nous indique que nous avons atteint une feuille de notre arbre d’expression.

Donc il y a cas possibles pour une expression `SymPy`. 

1. `.args` vide, dans ce cas là c'est un feuille dans un arbre
2. `.args` non-vide, et dans ce cas là c’est une branche d’un arbre

Dans le cas 2, l’expression peut être reconstruite à partir de ses attributs `.func` et `.args`.

<div class = 'alert alert-success'>
    
**La règle clé :**
    
Tout expression SymPy bien constituée doit soit :
    
- avoir un tuple vide dans sont attribut `.args`, ou bien
- satisfaire `expr == expr.func(*expr.args)`


</div>

<div class = 'alert alert-info'>

En Python, si `a` est un tuple, alors `f(*a)` signifie appelle la fonction `f` avec comme arguments les éléments de `a`,
*ie* `f(*(1, 2, 3))` revient à `f(1, 2, 3)`.

</div>

<div class = 'alert alert-success'>
    
Cette règle clé nous permet d’écrire des algorithmes simples qui traversent des arbres d’expression, les modifient, et  reconstruisent de nouvelles expressions.

</div>

### Traverser l’arbre

Armés de ce savoir, regardons comment on peut itérer et traverser un arbre d’expression.

La nature imbriquée des `.args` est parfaitement adaptée aux fonctions recurrentes.

Le cas de base sera un `.args` vide. Écrivons une fonction qui traverse une expression et `print()` tous les `args` à chaque niveau.

In [ ]:
def pre(expr):
    print(expr)
    for arg in expr.args:
        pre(arg) # appel à la fonction dans la fonction

In [ ]:
expr = x*y + 1
pre(expr)

Pouvez-vous deviner pourquoi nous avons appelé notre fonction `pre` ? 

Nous venons d’écrire une fonction qui traverse notre expression à l’endroit, `preorder` en anglais.

Essayez de créer une fonction `post()` qui la traverse dans l’ordre inverse.

In [ ]:
# à vous      

Traverser une expression de la sorte est si commun dans `SymPy` que les fonctions `preorder_traversal()` et `postorder_traversal()` sont données pour rendre ces traversées plus simples. On aurait pu réécrire notre algorithme pour `pre()` ainsi :

In [ ]:
for arg in preorder_traversal(expr):
    print(arg)

## Empêcher l’évaluation d’une expression

Il y a 2 manières d’empêcher l’évaluation d’une expression :

1. passer un argument `evaluate=False` quand vous construisez l’expression
2. créer un stopper d’évaluation en entourant l’expression avec `UnevaluatedExpr`.

Par exemple :

In [ ]:
x + x

In [ ]:
Add(x, x)

In [ ]:
Add(x, x, evaluate=False)

Si vous ne vous souvenez pas quelle classe correspond à l’expression que
vous voulez construire, utilisez `sympify()` sur une string :

In [ ]:
sympify("x + x", evaluate=False)

Notez que ce paramètre `evaluate=False` n’empêche pas l’évaluation d’expressions construites à partir de l’expression non-évaluée :

In [ ]:
expr = Add(x, x, evaluate=False)
expr

In [ ]:
expr + x

C’est là que la classe `UnevaluatedExpr` entre en scène. `UnevaluatedExpr` est une classe qui vous permet de créer une expression qui restera non-évaluée. Par non-évaluée, comprenez que cette expression ne sera pas combinée à d’autres autour d’elle dans des simplifications.

Exemple :

In [ ]:
expr = x + UnevaluatedExpr(x)
expr

In [ ]:
x + expr

Le $x$ qui reste tout seul est celui qui est passé par `UnevaluatedExpr()`.

Pour l’en libérer : `.doit()`

In [ ]:
(x + expr).doit()

Autres exemples :

In [ ]:
uexpr = UnevaluatedExpr(S.One*5/7)*UnevaluatedExpr(S.One*3/4)
uexpr

In [ ]:
x*UnevaluatedExpr(1/x)

Notez bien que `UnevaluatedExpr` ne peut pas empêcher l’évaluation d’une expression qui est passée comme argument à une fonction.

In [ ]:
expr1 = UnevaluatedExpr(x + x)
expr1

In [ ]:
expr2 = sympify('x + x', evaluate=False)
expr2

Souvenez-vous que `expr2` sera évaluée si incluse dans une autre expression. 

Combinez les deux techniques pour empêcher l’évaluation interne et externe :

In [ ]:
UnevaluatedExpr(sympify('x + x', evaluate=False)) + y

`UnevaluatedExpr` fonctionne avec les différents moteurs de rendu de `SymPy`, et peut être utilisée pour afficher le résultat sous différentes formes, par exemple en $LaTeX$ :

In [ ]:
uexpr = UnevaluatedExpr(S.One*5/7)*UnevaluatedExpr(S.One*3/4)

print(latex(uexpr))

À nouveau, pour laisser l’expression s’évaluer, utilisez la méthode `.doit()` :

In [ ]:
print(latex(uexpr.doit()))

C’est la fin de cette série de notebooks sur `SymPy`.

J’espère qu’ils vous ont été utiles.